# Setup

In [ ]:
%pip install astropy
%pip install git+https://github.com/pvigier/perlin-numpy
%pip install tensorflow_addons

In [ ]:
!git clone https://github.com/tomasmuzas/phd

In [1]:
from phd.Augmentations.common_code.augmentation_functions import *
from phd.Augmentations.common_code.get_dataset import *
from phd.Augmentations.common_code.models import *
from phd.Augmentations.common_code.training_loop import *
from phd.Augmentations.common_code.utils import *
import tensorflow as tf

training_config = {
  "ENABLE_DETERMINISM": False,
  "SEED": 17,
  "AUTO": tf.data.AUTOTUNE,
  "TRAIN_BATCH_SIZE": 1024,
  "TEST_BATCH_SIZE": 16,
  "NUMBER_OF_EPOCHS": 500,
  "EARLY_STOPPING_TOLERANCE": 10,
  "LEARNING_RATE": 1e-4,
  "IMAGE_SIZE": 128,
  "SHUFFLE_BUFFER": 300000,
  "TPU": True,
  "AUGMENTATIONS_ROTATE": True,
  "AUGMENTATIONS_FLIP_HORIZONTALLY": True,
  "AUGMENTATIONS_FLIP_VERTICALLY": True,
  "AUGMENTATIONS_ZOOM": True,
  "AUGMENTATIONS_RANDOM_NOISE": False,
  "AUGMENTATIONS_CENTER_NOISE": False,
  "AUGMENTATIONS_OUTSIDE_NOISE": False,
  "AUGMENTATIONS_PERLIN_CENTER_NOISE": False,
  "USE_ADABELIEF_OPTIMIZER": False,
  "GALAXY_DATASET": "SDSS",
  "EXPERIMENT_DESCRIPTION": "conference/basic_augmentations"
}

training_config["LOCAL_GCP_PATH_BASE"] = f"gcs/{training_config['GALAXY_DATASET']}"
training_config["REMOTE_GCP_PATH_BASE"] = f"gs://tomasmuzas-phd-datasets/{training_config['GALAXY_DATASET']}"


if training_config["ENABLE_DETERMINISM"]:
  print(f"Enabling determinism. Seed: {training_config.SEED}.")
  tf.keras.utils.set_random_seed(training_config.SEED)
  tf.config.experimental.enable_op_determinism()
else:
  print("No determinism enabled.")

No determinism enabled.


In [2]:
from google.colab import auth

auth.authenticate_user()
reset_tpu(training_config)

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse
!mkdir gcs
!gcsfuse --implicit-dirs tomasmuzas-phd-datasets gcs

# Training

In [ ]:
models = [
    {'name': 'cavanagh', 'func': Cavanagh, 'starting_fold': 1},
    {'name': 'dieleman', 'func': Dieleman, 'starting_fold': 1},
    {'name': 'resnet50', 'func': create_ResNet50, 'starting_fold': 1},
    {'name': 'newsimple', 'func': SimpleModel, 'starting_fold': 1}
]

perform_training(models, training_config)